In [14]:
import pandas as pd
from pathlib import Path
import numpy as np
import scipy.stats as stats

#Path to CSV
transportcsv = Path("Resources/nanda_transit_zcta.csv")
collegereadycsv = Path("Resources/CollegeReadiness_2017_2018_byZip_20181107.csv")
freelunchcsv = Path("Resources/FreeAndReducedLunch_Fall207_ByZIP_20181105.csv")
graduationcsv = Path("Resources/GraduationRates_2016_2017_byZip__20181019.csv")

In [15]:
#Read in CSV to dataframes
transport_data = pd.read_csv(transportcsv)
collegeready_data = pd.read_csv(collegereadycsv)
freelunch_data = pd.read_csv(freelunchcsv)
graduation_data = pd.read_csv(graduationcsv)

In [16]:
#Rename column so it can be merged
collegeready_data = collegeready_data.rename(columns={'ZCTA':'ZCTA5CE10'})
collegeready_data

,OBJECTID,GEOID10,MathNumAssessed,MathNumReady,AllSbjtNumReady,AllSbjtNumAssessed,EBRWNumAssessed,EBRWNumReady,FinalMathAveScore,FinalAllSbjtAveScore,FinalEWBRWAveScore,MathPctReady,AllSbjtPctReady,EBRWPctReady,Shape__Area,Shape__Length,ZCTA5CE10
0,1,2649883,25.0,0.0,0.0,25.0,25.0,6.0,404.800000,841.600000,436.800000,0.000000,0.000000,0.240000,7.789234e+09,5.799556e+05,49883
1,2,2649878,34.0,12.0,11.0,34.0,34.0,23.0,484.400000,983.500000,499.100000,0.352941,0.323529,0.676471,1.404697e+10,9.217413e+05,49878
2,3,2649855,285.0,115.0,107.0,285.0,285.0,181.0,507.962105,1033.437193,525.387368,0.403509,0.375439,0.635088,6.747444e+09,1.162098e+06,49855
3,4,2649408,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.995690e+09,3.517039e+05,49408
4,5,2649010,168.0,52.0,49.0,168.0,168.0,87.0,477.414286,967.171429,489.678571,0.309524,0.291667,0.517857,4.615699e+09,5.311028e+05,49010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,985,2648063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.032694e+09,1.285146e+05,48063
985,986,2648414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.262379e+08,1.955999e+05,48414
986,987,2648701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.319999e+09,2.687271e+05,48701
987,988,2648733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.397144e+09,2.535510e+05,48733


In [17]:
#Merge of 3 CSVs into a dataframe on zip
merge1_df = pd.merge(freelunch_data, graduation_data, on=["ZCTA5CE10"])
merge2_df = pd.merge(merge1_df, collegeready_data, on=["ZCTA5CE10"])

In [18]:
#Filter zip from national to zips listed in other merged dataframe (i.e. zips in michigan)
filter_data = transport_data[transport_data["zcta19"].isin(merge2_df["ZCTA5CE10"])]
filter_data

,zcta19,zcta_area,count_ntm_stops,totpop13_17,stops_per_capita,stops_per_sqmile
15734,48001,23.655580,0,11886.0,0.000000,0.000000
15736,48003,33.083480,0,6147.0,0.000000,0.000000
15737,48005,36.561310,0,5476.0,0.000000,0.000000
15739,48009,5.055899,2,20836.0,0.095988,0.395578
15740,48014,36.796970,0,4174.0,0.000000,0.000000
...,...,...,...,...,...,...
16701,49945,182.779200,0,2501.0,0.000000,0.000000
16702,49946,297.685800,0,3898.0,0.000000,0.000000
16707,49953,421.172400,0,2493.0,0.000000,0.000000
16717,49968,252.539400,0,2072.0,0.000000,0.000000


In [19]:
#Merge filtered transporation data into other data frames for a final merge
transport_data = transport_data.rename(columns={'zcta19':'ZCTA5CE10'})
final_merge_df = pd.merge(merge2_df, transport_data, on='ZCTA5CE10')

In [20]:
#Remove columns not needed
final_merge_df = final_merge_df[["ZCTA5CE10", "GEOID10_x", "FreeEligible", "ReducedEligible", "NotEligible",
                                 "FreeReducedEligible", "Total", "PCT_Free", "PCT_Reduced", "PCT_FreeReduced",
                                 "CohortCnt", "GradCnt", "GradRate", "MathNumAssessed", "AllSbjtNumAssessed",
                                 "EBRWNumAssessed", "MathNumReady", "AllSbjtNumReady", "EBRWNumReady",
                                 "FinalMathAveScore", "FinalAllSbjtAveScore", "FinalEWBRWAveScore", "MathPctReady",
                                 "AllSbjtPctReady", "EBRWPctReady", 'count_ntm_stops', 'stops_per_capita', 'stops_per_sqmile']]
#Rename columns
final_merge_df = final_merge_df.rename(columns={"GEOID10_x":"GEOID10",
                                                "FreeReducedEligible":"Free_Reduced_Eligible_Total",
                                                "Total":"Total_Students"})
final_merge_df

,ZCTA5CE10,GEOID10,FreeEligible,ReducedEligible,NotEligible,Free_Reduced_Eligible_Total,Total_Students,PCT_Free,PCT_Reduced,PCT_FreeReduced,...,EBRWNumReady,FinalMathAveScore,FinalAllSbjtAveScore,FinalEWBRWAveScore,MathPctReady,AllSbjtPctReady,EBRWPctReady,count_ntm_stops,stops_per_capita,stops_per_sqmile
0,49878,2649878,142.0,23.0,190.0,165.0,355.0,0.400000,0.064789,0.464789,...,23.0,484.400000,983.500000,499.100000,0.352941,0.323529,0.676471,0,0.000000,0.000000
1,49855,2649855,985.0,151.0,2437.0,1136.0,3573.0,0.275679,0.042261,0.317940,...,181.0,507.962105,1033.437193,525.387368,0.403509,0.375439,0.635088,0,0.000000,0.000000
2,49408,2649408,273.0,68.0,99.0,341.0,440.0,0.620455,0.154545,0.775000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
3,49010,2649010,1317.0,224.0,1162.0,1541.0,2703.0,0.487236,0.082871,0.570107,...,87.0,477.414286,967.171429,489.678571,0.309524,0.291667,0.517857,0,0.000000,0.000000
4,49055,2649055,2.0,0.0,7.0,2.0,9.0,0.222222,0.000000,0.222222,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,48124,2648124,4149.0,277.0,2394.0,4426.0,6820.0,0.608358,0.040616,0.648974,...,445.0,497.590599,1004.114169,506.485286,0.369210,0.350136,0.606267,1,0.034327,0.112365
525,48183,2648183,1667.0,164.0,3580.0,1831.0,5411.0,0.308076,0.030309,0.338385,...,149.0,514.600000,1028.500000,513.900000,0.472574,0.443038,0.628692,0,0.000000,0.000000
526,48240,2648240,3349.0,196.0,941.0,3545.0,4486.0,0.746545,0.043691,0.790236,...,47.0,406.458367,830.209388,423.744898,0.081633,0.065306,0.191837,0,0.000000,0.000000
527,48125,2648125,2626.0,192.0,791.0,2818.0,3609.0,0.727625,0.053200,0.780826,...,101.0,474.872917,960.092188,485.264063,0.265625,0.255208,0.526042,0,0.000000,0.000000


In [21]:
#Drop NaN values
clean_merge_df = final_merge_df.dropna()

In [22]:
#Remove zipcodes with no public transport data, aka 0 stops.
final_clean_df = clean_merge_df[clean_merge_df['count_ntm_stops'] != 0]
final_clean_df

,ZCTA5CE10,GEOID10,FreeEligible,ReducedEligible,NotEligible,Free_Reduced_Eligible_Total,Total_Students,PCT_Free,PCT_Reduced,PCT_FreeReduced,...,EBRWNumReady,FinalMathAveScore,FinalAllSbjtAveScore,FinalEWBRWAveScore,MathPctReady,AllSbjtPctReady,EBRWPctReady,count_ntm_stops,stops_per_capita,stops_per_sqmile
18,48917,2648917,2857.0,419.0,1862.0,3276.0,5138.0,0.556053,0.081549,0.637602,...,136.0,431.737912,888.779945,457.082967,0.140110,0.131868,0.373626,77,2.393610,3.098086
22,48864,2648864,2186.0,327.0,4850.0,2513.0,7363.0,0.296890,0.044411,0.341301,...,446.0,517.842420,1051.718950,533.870991,0.451895,0.440233,0.650146,86,3.796570,3.955903
25,49008,2649008,2291.0,190.0,1083.0,2481.0,3564.0,0.642817,0.053311,0.696128,...,0.0,377.435366,778.524390,401.089024,0.000000,0.000000,0.000000,92,5.543170,13.630350
26,49002,2649002,1478.0,230.0,2769.0,1708.0,4477.0,0.330132,0.051374,0.381505,...,446.0,505.665653,1035.593921,529.928267,0.392097,0.379939,0.677812,53,2.601740,2.592890
30,49544,2649544,971.0,193.0,1411.0,1164.0,2575.0,0.377087,0.074951,0.452039,...,125.0,485.395283,986.886792,501.500472,0.325472,0.306604,0.589623,21,2.108430,1.001814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,48234,2648234,2209.0,52.0,208.0,2261.0,2469.0,0.894694,0.021061,0.915755,...,0.0,380.200000,777.700000,397.500000,0.000000,0.000000,0.000000,293,8.481200,37.404350
521,48210,2648210,7455.0,114.0,736.0,7569.0,8305.0,0.897652,0.013727,0.911379,...,104.0,438.858859,887.266366,448.407508,0.159159,0.117117,0.312312,131,4.344220,25.993740
522,48202,2648202,3087.0,332.0,933.0,3419.0,4352.0,0.709329,0.076287,0.785616,...,82.0,416.437716,854.894810,438.420069,0.089965,0.065744,0.283737,178,10.681700,53.505260
523,48235,2648235,4813.0,212.0,1318.0,5025.0,6343.0,0.758789,0.033423,0.792212,...,222.0,460.555022,934.710480,474.139083,0.213974,0.196507,0.484716,225,5.008790,35.994660


In [24]:
#Define a function to remove outliers using z-score
def remove_outliers(final_clean_df, threshold = 3):
    zscores = stats.zscore(final_clean_df)
    abs_zscores = abs(zscores)
    filtered_entries = (abs_zscores < threshold).all(axis=1)
    return final_clean_df[filtered_entries]

#Apply function to DataFrame
cleaned_df = remove_outliers(final_clean_df)
cleaned_df

,ZCTA5CE10,GEOID10,FreeEligible,ReducedEligible,NotEligible,Free_Reduced_Eligible_Total,Total_Students,PCT_Free,PCT_Reduced,PCT_FreeReduced,...,EBRWNumReady,FinalMathAveScore,FinalAllSbjtAveScore,FinalEWBRWAveScore,MathPctReady,AllSbjtPctReady,EBRWPctReady,count_ntm_stops,stops_per_capita,stops_per_sqmile
18,48917,2648917,2857.0,419.0,1862.0,3276.0,5138.0,0.556053,0.081549,0.637602,...,136.0,431.737912,888.779945,457.082967,0.140110,0.131868,0.373626,77,2.393610,3.098086
22,48864,2648864,2186.0,327.0,4850.0,2513.0,7363.0,0.296890,0.044411,0.341301,...,446.0,517.842420,1051.718950,533.870991,0.451895,0.440233,0.650146,86,3.796570,3.955903
25,49008,2649008,2291.0,190.0,1083.0,2481.0,3564.0,0.642817,0.053311,0.696128,...,0.0,377.435366,778.524390,401.089024,0.000000,0.000000,0.000000,92,5.543170,13.630350
26,49002,2649002,1478.0,230.0,2769.0,1708.0,4477.0,0.330132,0.051374,0.381505,...,446.0,505.665653,1035.593921,529.928267,0.392097,0.379939,0.677812,53,2.601740,2.592890
30,49544,2649544,971.0,193.0,1411.0,1164.0,2575.0,0.377087,0.074951,0.452039,...,125.0,485.395283,986.886792,501.500472,0.325472,0.306604,0.589623,21,2.108430,1.001814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,48234,2648234,2209.0,52.0,208.0,2261.0,2469.0,0.894694,0.021061,0.915755,...,0.0,380.200000,777.700000,397.500000,0.000000,0.000000,0.000000,293,8.481200,37.404350
521,48210,2648210,7455.0,114.0,736.0,7569.0,8305.0,0.897652,0.013727,0.911379,...,104.0,438.858859,887.266366,448.407508,0.159159,0.117117,0.312312,131,4.344220,25.993740
522,48202,2648202,3087.0,332.0,933.0,3419.0,4352.0,0.709329,0.076287,0.785616,...,82.0,416.437716,854.894810,438.420069,0.089965,0.065744,0.283737,178,10.681700,53.505260
523,48235,2648235,4813.0,212.0,1318.0,5025.0,6343.0,0.758789,0.033423,0.792212,...,222.0,460.555022,934.710480,474.139083,0.213974,0.196507,0.484716,225,5.008790,35.994660


In [11]:
final_clean_df.to_csv('final_df.csv', index=False)